Explore the DataSet in this Notebook

In [19]:
import pandas as pd
import os
import glob

In [20]:
r2r_dataset = pd.read_json("Dataset/R2R_train.json")

r2r_dataset = r2r_dataset[r2r_dataset['scan'] == "17DRP5sb8fy"]

r2r_dataset.head()

,distance,scan,path_id,path,heading,instructions
40,6.06,17DRP5sb8fy,3399,"[10c252c90fa24ef3b698c6f54d984c5c, 77a1a11978b...",3.506,[Walk past the large picture and chair. Walk p...
357,6.16,17DRP5sb8fy,5894,"[db145474a5fa476d95c2cc7f09e7c83a, 5b9b2794954...",4.610,[Walk to the foot of the bed and to a little s...
578,8.83,17DRP5sb8fy,2505,"[ee59d6b5e5da4def9fe85a8ba94ecf25, 5e9f4f86545...",0.666,[Go straight and slightly right toward the gla...
1000,7.68,17DRP5sb8fy,6066,"[e693b5de8ad84d4cb61a79ece2e66d11, 28db29e8c72...",0.922,"[Exit the bathroom, cross the room passed the ..."
1036,5.30,17DRP5sb8fy,2571,"[d65b6505904448d1940e679c9a098047, 701f7128272...",1.616,[Go stand behind the middle kitchen island cha...


In [27]:
path = r2r_dataset.iloc[0]['path']

print(path)
# Specify the directory where your images are stored
folder_path = 'Dataset/17DRP5sb8fy/matterport_color_images'

# Specify the word the image names should start with
start_word = path[0]

# Construct the search pattern
pattern = os.path.join(folder_path, f'{start_word}*')

# Get a list of all image filenames that start with the specific word
image_files = glob.glob(pattern)

# Print the filenames (only the names, not the full paths)
image_names = [os.path.basename(image) for image in image_files]
print(image_names[5:8])

chat_template = "You are an LLM Planner, design to take in an end goal,and summary of what has happened before.This is given to you in JSON format. You are also given multiple images, which tell you about the current scene. Use these images, the end goal and the summary to give a sentence on what we should do next to acheive the end goal.###Human: "+(" ").join(["<image>" for image_name in image_names])+"\n<prompt>###Assistant:"

print(chat_template.split("###"))

['10c252c90fa24ef3b698c6f54d984c5c', '77a1a11978b04e9cbf74914c98578ab8', 'b185432bf33645aca813ac2a961b4140', '5e9f4f8654574e699480e90ecdd150c8', '08c774f20c984008882da2b8547850eb', 'da5fa65c13e643719a20cbb818c9a85d']
['10c252c90fa24ef3b698c6f54d984c5c_i0_5.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_0.jpg', '10c252c90fa24ef3b698c6f54d984c5c_i1_1.jpg']
['You are an LLM Planner, design to take in an end goal,and summary of what has happened before.This is given to you in JSON format. You are also given multiple images, which tell you about the current scene. Use these images, the end goal and the summary to give a sentence on what we should do next to acheive the end goal.', 'Human: <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image> <image>\n<prompt>', 'Assistant:']
